In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import matplotlib.image as img
from PIL import Image
import cv2
from pathlib import Path
from typing import Dict, List, Tuple
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import timm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
data = np.load("../Resize_Dataset.npz")
print(data.files)

['label', 'image']


In [ ]:
arrays = data['image']
label = data['label']
# print(arrays.shape)
arrays = np.reshape(arrays,(-1,65*65))
combine_array = np.concatenate([label,arrays],axis=1)
# print(combine_array.shape)

np.random.shuffle(combine_array)

# 設定每個集合的大小
total_rows = combine_array.shape[0]  # 總行數
train_size = int(total_rows * 0.7)  # 訓練集大小 (70%)
val_size = int(total_rows * 0.15)   # 驗證集大小 (15%)
test_size = total_rows - train_size - val_size  # 測試集大小 (剩下的行數)

# 分割數據集
train_set = combine_array[:train_size]  # 前 70% 為訓練集
val_set = combine_array[train_size:train_size + val_size]  # 接下來的 15% 為驗證集
test_set = combine_array[train_size + val_size:]  # 剩下的 15% 為測試集

print("訓練集筆數:", train_set.shape[0])
print("驗證集筆數:", val_set.shape[0])
print("測試集筆數:", test_set.shape[0])

train_X = train_set[:,8: ]
train_Y = train_set[:, :8]
val_X = val_set[:,8: ]
val_Y = val_set[:, :8]
test_X = test_set[:,8: ]
test_Y = test_set[:, :8]

train_X = np.reshape(train_X,(-1,65,65))
val_X = np.reshape(val_X,(-1,65,65))
test_X = np.reshape(test_X,(-1,65,65))

print(train_X.shape)

# 看各 label 個數
print(pd.DataFrame(train_Y).value_counts())

def decode_to_38bit(input_ndarray):
    mapping = {
        "00000000": 0,
        "10000000": 1,
        "01000000": 2,
        "00100000": 3,
        "00010000": 4,
        "00001000": 5,
        "00000100": 6,
        "00000010": 7,
        "00000001": 8,
        "10100000": 9,
        "10010000": 10,
        "10001000": 11,
        "10000010": 12,
        "01100000": 13,
        "01010000": 14,
        "01001000": 15,
        "01000010": 16,
        "00101000": 17,
        "00100010": 18,
        "00011000": 19,
        "00010010": 20,
        "00001010": 21,
        "10101000": 22,
        "10100010": 23,
        "10011000": 24,
        "10010010": 25,
        "10001010": 26,
        "01101000": 27,
        "01100010": 28,
        "01011000": 29,
        "01010010": 30,
        "01001010": 31,
        "00101010": 32,
        "00011010": 33,
        "10101010": 34,
        "10011010": 35,
        "01101010": 36,
        "01011010": 37
    }
    n = input_ndarray.shape[0]  # 行數
    decoded_ndarray = np.zeros(n, dtype=int)
    # 逐行解碼
    for i, row in enumerate(input_ndarray):
        # 將每一行轉換為字典的鍵，並獲得對應的索引
        input_8bit = ''.join(row.astype(str))  # 把整數轉為字串並拼接
        number = mapping[input_8bit]  # 獲取對應的數字索引
        decoded_ndarray[i] = number
    return decoded_ndarray

# label 轉成 one hot encoding
train_Y = decode_to_38bit(train_Y)
val_Y = decode_to_38bit(val_Y)
test_Y = decode_to_38bit(test_Y)

訓練集筆數: 79831
驗證集筆數: 17106
測試集筆數: 17108
(79831, 65, 65)
0  1  2  3  4  5  6  7
1  0  1  0  0  0  1  0    4171
0  1  1  0  0  0  1  0    2162
1  0  0  0  1  0  0  0    2162
0  1  1  0  1  0  1  0    2159
1  0  0  0  0  0  0  0    2149
0  1  0  0  1  0  1  0    2147
            0  0  1  0    2146
1  0  0  0  0  0  1  0    2136
0  0  0  1  0  0  0  0    2118
   1  1  0  0  0  0  0    2117
1  0  1  0  0  0  0  0    2113
0  0  1  0  0  0  1  0    2111
   1  0  0  0  0  0  0    2108
         1  0  0  1  0    2106
         0  1  0  0  0    2104
   0  0  1  1  0  0  0    2102
            0  0  1  0    2101
1  0  0  1  0  0  0  0    2099
      1  0  1  0  0  0    2099
      0  0  1  0  1  0    2099
0  0  0  0  0  0  1  0    2098
   1  0  1  1  0  0  0    2096
      1  0  1  0  0  0    2093
   0  0  0  1  0  0  0    2092
      1  0  1  0  1  0    2091
   1  0  1  1  0  1  0    2090
   0  1  0  1  0  0  0    2088
            0  0  0  0    2088
1  0  0  1  1  0  0  0    2087
                  1  0 

In [ ]:
# 自定義一個簡單的 Dataset 類
class MyDataset(Dataset):
    def __init__(self,mydata,mylabel):
        # 定義數據
        self.data = mydata
        self.labels = mylabel

    def __len__(self):
        # 返回數據集的大小
        return len(self.data)

    def __getitem__(self, index):
        # 返回指定 index 的數據和標籤
        return self.data[index], self.labels[index]

# --- 步驟 1: 定義一個更穩健的 CategoricalCNN 模型 ---
# 這個版本能更可靠地自動獲取骨幹網路的輸出特徵數量
class CategoricalCNN(nn.Module):
    """
    專為二維類別資料（如晶圓圖）設計的 CNN。
    它使用嵌入層從類別創建特徵向量，然後將其饋送到標準的 CNN 骨幹網路。
    """
    def __init__(self, num_classes, num_categories=3, embedding_dim=16, backbone_name='efficientnet_b0'):
        super().__init__()
        self.num_classes = num_classes
        self.num_categories = num_categories
        self.embedding_dim = embedding_dim

        # 1. 嵌入層，用於學習晶圓圖中各個類別的向量表示
        # 輸入是類別索引 (0, 1, 2)，輸出是密集向量
        self.embedding = nn.Embedding(num_embeddings=num_categories, embedding_dim=embedding_dim)

        # 2. 從 timm 中選擇一個現代化的 CNN 骨幹網路
        self.backbone = timm.create_model(
            backbone_name,
            pretrained=False,        # 盡可能使用預訓練權重
            features_only=True,     # 我們只需要特徵提取部分
            in_chans=embedding_dim  # 關鍵：將骨幹網路的輸入通道數修改為嵌入維度
        )

        # 3. 使用 timm 的 feature_info 來安全地獲取骨幹網路的輸出特徵維度
        num_features = self.backbone.feature_info.channels(-1)

        # 4. 分類頭
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(num_features, num_classes)

    def forward(self, x):
        # x 的輸入形狀: (batch_size, height, width)，其值為整數類別
        # `embedding` 層需要 LongTensor
        x = x.long()

        # 通過嵌入層處理
        # 輸出形狀: (batch_size, height, width, embedding_dim)
        embedded_x = self.embedding(x)

        # 調整維度以匹配 CNN 的輸入格式 (N, C, H, W)
        # 輸出形狀: (batch_size, embedding_dim, height, width)
        embedded_x = embedded_x.permute(0, 3, 1, 2)

        # 從骨幹網路提取特徵
        # features_only=True 的輸出是一個包含不同階段特徵圖的列表
        features = self.backbone(embedded_x)

        # 我們使用最後一個、包含最豐富資訊的特徵圖進行分類
        last_feature_map = features[-1]

        # 應用全域池化和最終的分類層
        pooled_features = self.global_pool(last_feature_map).flatten(1)
        output = self.classifier(pooled_features)

        return output

In [ ]:
# 將原始的 one-hot 編碼資料賦予新的變數名稱，以符合後續程式碼的命名
train_Y_onehot = train_Y
val_Y_onehot = val_Y
test_Y_onehot = test_Y

# 使用 OpenCV 的 resize 函數進行調整
def resize_ndarray(input_array, target_height, target_width):
    return cv2.resize(input_array, (target_width, target_height), interpolation=cv2.INTER_NEAREST)

IMG_SIZE = 65
print(f"正在將晶圓圖大小從 (52, 52) 調整為 ({IMG_SIZE}, {IMG_SIZE})...")
resized_train_X = np.array([resize_ndarray(img, IMG_SIZE, IMG_SIZE) for img in tqdm(train_X, desc="調整訓練集大小")])
resized_val_X = np.array([resize_ndarray(img, IMG_SIZE, IMG_SIZE) for img in tqdm(val_X, desc="調整驗證集大小")])
resized_test_X = np.array([resize_ndarray(img, IMG_SIZE, IMG_SIZE) for img in tqdm(test_X, desc="調整測試集大小")])
print("大小調整完成。")

# --- 步驟 3: 更新 Dataset, Config 和訓練函式 ---
# 更新後的 Dataset，確保返回正確的資料類型
class WaferMapDataset(Dataset):
    def __init__(self, mydata, mylabel):
        # 將資料轉換為 LongTensor 以便嵌入層使用
        self.data = torch.from_numpy(mydata).long()
        # 將標籤轉換為 LongTensor 以便損失函式使用
        self.labels = torch.from_numpy(mylabel).long()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # 返回指定 index 的數據和標籤
        return self.data[index], self.labels[index]

# 更新後的 Config
class Config:
    def __init__(self):
        self.seed = 42
        self.image_size = 52
        self.batch_size = 16  # 如果記憶體不足 (CUDA out of memory)，可以降低此值
        self.num_workers = 2
        self.num_epochs = 30
        self.learning_rate = 5e-5
        # 建議從一個較小的模型開始，以加快實驗速度
        self.model_name = 'mobilenet_edgetpu_v2_l'
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.output_dir = Path("./save")
        self.model_path = self.output_dir / f"{self.model_name}.pth"

        # CategoricalCNN 所需的新參數
        # 您的晶圓圖資料中的獨特值數量 (例如 0, 1, 2)
        # 如果不確定，可以執行 np.unique(train_X) 來檢查
        self.num_categories = 38
        # 學習到的類別向量維度，16 是一個不錯的起點
        self.embedding_dim = 16

        # 類別和標籤
        self.num_classes = 38
        self.categories = [str(i) for i in range(self.num_classes)] # 用於分類報告

# 您的 train_epoch 和 validate 函式無需修改，這裡為了完整性而包含進來
def train_epoch(model, train_loader, criterion, optimizer, device, epoch):
    model.train()
    total_loss = 0
    with tqdm(train_loader, desc=f'Epoch {epoch + 1} - 訓練中') as pbar:
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    return total_loss / len(train_loader)


@torch.no_grad()
def validate(model, val_loader, criterion, device, desc='驗證中'):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with tqdm(val_loader, desc=desc) as pbar:
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            accuracy = 100 * correct / total
            pbar.set_postfix({'loss': f'{loss.item():.4f}', 'accuracy': f'{accuracy:.2f}%'})
    return total_loss / len(val_loader), 100 * correct / total

# --- 步驟 4: 更新主函式以使用新模型和資料 ---
def main():
    config = Config()
    config.output_dir.mkdir(exist_ok=True)
    print(f"使用設備: {config.device}")
    print(f"模型將儲存至: {config.model_path}")

    # 設定隨機種子以確保可重現性
    torch.manual_seed(config.seed)
    np.random.seed(config.seed)
    random.seed(config.seed)

    # 初始化 Dataset 和 DataLoader
    train_dataset = WaferMapDataset(resized_train_X, train_Y)
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)

    val_dataset = WaferMapDataset(resized_val_X, val_Y)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, num_workers=config.num_workers)

    test_dataset = WaferMapDataset(resized_test_X, test_Y)
    test_loader = DataLoader(test_dataset, batch_size=config.batch_size, num_workers=config.num_workers)

    # --- 關鍵變更：實例化新的 CategoricalCNN 模型 ---
    model = CategoricalCNN(
        num_classes=config.num_classes,
        num_categories=config.num_categories,
        embedding_dim=config.embedding_dim,
        backbone_name=config.model_name
    ).to(config.device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.num_epochs)

    best_val_acc = 0
    for epoch in range(config.num_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, config.device, epoch)
        val_loss, val_acc = validate(model, val_loader, criterion, config.device, desc=f'Epoch {epoch + 1} - 驗證中')
        scheduler.step()

        print(f'\nEpoch {epoch + 1}:')
        print(f'訓練損失: {train_loss:.4f}')
        print(f'驗證損失: {val_loss:.4f}, 驗證準確率: {val_acc:.2f}%')

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), config.model_path)
            print(f'儲存最佳模型，驗證準確率: {val_acc:.2f}%')

    print("\n在測試集上評估最佳模型...")
    model.load_state_dict(torch.load(config.model_path))
    test_loss, test_acc = validate(model, test_loader, criterion, config.device, desc='測試中')
    print(f'\n測試準確率: {test_acc:.2f}%')

    print("\n正在生成分類報告...")
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc='預測中'):
            images = images.to(config.device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())

    report = classification_report(all_labels, all_preds, target_names=config.categories, digits=4)
    print('\n分類報告:')
    print(report)

    with open(config.output_dir / 'classification_report.txt', 'w') as f:
        f.write(report)

main()

正在將晶圓圖大小從 (52, 52) 調整為 (65, 65)...


調整測試集大小: 100%|██████████| 17108/17108 [00:00<00:00, 129354.69it/s]


大小調整完成。
使用設備: cuda
模型將儲存至: /content/drive/MyDrive/Colab Notebooks/save/mobilenet_edgetpu_v2_l.pth


Epoch 1 - 驗證中: 100%|██████████| 1070/1070 [00:24<00:00, 43.37it/s, loss=1.0512, accuracy=76.27%]



Epoch 1:
訓練損失: 2.0250
驗證損失: 0.7346, 驗證準確率: 76.27%
儲存最佳模型，驗證準確率: 76.27%


...

Epoch 8 - 驗證中: 100%|██████████| 1070/1070 [00:24<00:00, 43.73it/s, loss=1.2007, accuracy=92.12%]



Epoch 8:
訓練損失: 0.1458
驗證損失: 0.2581, 驗證準確率: 92.12%


Epoch 9 - 驗證中: 100%|██████████| 1070/1070 [00:24<00:00, 44.21it/s, loss=1.1860, accuracy=91.08%]



Epoch 9:
訓練損失: 0.1176
驗證損失: 0.3242, 驗證準確率: 91.08%


Epoch 10 - 訓練中:  34%|███▍      | 1711/4990 [01:41<02:53, 18.95it/s, loss=0.0090]